<h1>Dimensionality Reduction with TICA<h1>

In [ ]:
from msmbuilder.io import load_trajs, save_trajs, save_generic
from msmbuilder.decomposition import tICA
import numpy as np
import os

def run_tica(featdir,no_of_traj,ticadir,index_dir=None):
    """
    Running tlCA based on feature selection from Spectral oASIS and save the output

    Parameters
    ----------
    featdir : string
        Directory of full features
    no_of_traj : int
        Number of trajectorues from MD simulation
    ticadir : string
        Directory for saving tlCA output

    Optional Parameters
    -------------------
    index_dir: str, Default=ticadir
        Directory for the column index from Spectral oASIS
    """
    #Setting up the parameters
    no_of_features=[20,24,28]
    tica_lagtime=[[2,4,6],[2,4,6],[2,4,6],[2,4,6]] # the tlCA lag time when running tlCA with different feature size
    #Load Dat
    pairwise_distance=[]
    for i in range(no_of_traj):
        temp=np.load("{}features/{}.npy".format(featdir,i))
        pairwise_distance.append(temp)
    pairwise_distance=np.array(pairwise_distance)


    for n,nth in enumerate(no_of_features):
        for m in range(0,len(tica_lagtime[n])):
            ftrajs={}
            columns=np.loadtxt("{}{}/feature_column{}_ticalag_{}.txt".format(index_dir,tica_lagtime[n][m],
                                                                             nth,tica_lagtime[n][m]),dtype=int)
            for i in range(len(pairwise_distance)):
                ftrajs[i]=pairwise_distance[i][:,columns]
            tica = tICA(n_components=10, lag_time=tica_lagtime[n][m], kinetic_mapping=True)
    ## Fit
            tica.fit(ftrajs.values())
    ## Transform
            ttrajs = {}
            for k, v in ftrajs.items():
                ttrajs[k] = tica.partial_transform(v)
            os.system("mkdir -p {}{}/tica_lag{}".format(ticadir,nth,int(tica_lagtime[n][m])))
            for keys in ttrajs:
                np.save("{}{}/tica_lag{}/{}.npy".format(ticadir,nth,int(tica_lagtime[n][m]),keys),ttrajs[keys])
            save_generic(tica, "{}{}/tica_lag{}/tica.pickl".format(ticadir,nth,int(tica_lagtime[n][m])))
            
ticadir="./TICA/"
featdir="./Featurization/"
spectraldir="./SpectralOasis/"
no_of_traj=100
run_tica(featdir,no_of_traj,ticadir,index_dir=spectraldir)